In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import joblib

2025-10-12 10:04:06.059399: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-12 10:04:06.059427: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-12 10:04:06.059448: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-12 10:04:06.065514: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-12 10:04:16.831481: W tensorflow/compiler/

In [4]:
df_churn = pd.read_csv("../src/dataset_churn.csv")

In [5]:
df_churn.columns

Index(['CustomerId', 'CreditScore', 'Geography', 'Gender', 'Age', 'Tenure',
       'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'days_since_last_tx', 'txs_avg_amount', 'amount_std',
       'avg_cashout_amount', 'ratio_recent_vs_past_txs',
       'ratio_recent_vs_past_amount', 'ratio_cashouts', 'ratio_transfers',
       'inflation_pressure', 'days_since_last_ss', 'total_ss_past30d',
       'total_ss_past90d', 'avg_ss_per_wk', 'avg_ss_duration_min',
       'std_ss_duration_min', 'ratio_ss_time_recent_vs_past',
       'ratio_events_sessios', 'ratio_failed_ss', 'total_opened_push',
       'Churn'],
      dtype='object')

In [23]:
features = ['CreditScore', 'Age', 'Tenure',
       'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'days_since_last_tx', 'txs_avg_amount', 'amount_std',
       'avg_cashout_amount', 'ratio_recent_vs_past_txs',
       'ratio_recent_vs_past_amount', 'ratio_cashouts', 'ratio_transfers',
       'inflation_pressure', 'days_since_last_ss', 'total_ss_past30d',
       'total_ss_past90d', 'avg_ss_per_wk', 'avg_ss_duration_min',
       'std_ss_duration_min', 'ratio_ss_time_recent_vs_past',
       'ratio_events_sessios', 'ratio_failed_ss', 'total_opened_push']

In [24]:
X = df_churn[features]

In [25]:
y = df_churn[['Churn']]

In [26]:
print(X.shape)
print(y.shape)

(9455, 27)
(9455, 1)


In [27]:
X = pd.get_dummies(X, drop_first=True)

In [28]:
scaler = StandardScaler()

In [29]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler , features)
    ]
)

In [30]:
X_processed = preprocessor.fit_transform(X)

In [31]:
joblib.dump(preprocessor, "pipeline_churn.pkl") 

['pipeline_churn.pkl']

In [33]:
X_processed.shape

(9455, 27)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X_processed ,y, test_size=0.2, random_state=42)

In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  
    Dropout(0.3), 
    Dense(32, activation='relu'), 
    Dense(1, activation='sigmoid')  
])

2025-10-12 10:09:43.652655: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-10-12 10:09:46.836998: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-10-12 10:09:46.837585: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [37]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                1792      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 3905 (15.25 KB)
Trainable params: 3905 (15.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [39]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32)

Epoch 1/50


2025-10-12 10:10:13.190970: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x718e94559c40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-10-12 10:10:13.191026: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2025-10-12 10:10:13.400570: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-10-12 10:10:15.468825: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8907
2025-10-12 10:10:15.782690: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


190/190 [==============================] - 11s 4ms/step - loss: 0.4726 - accuracy: 0.7845 - val_loss: 0.3688 - val_accuracy: 0.8420
Epoch 2/50
190/190 [==============================] - 1s 3ms/step - loss: 0.3626 - accuracy: 0.8528 - val_loss: 0.3316 - val_accuracy: 0.8612
Epoch 3/50
190/190 [==============================] - 1s 3ms/step - loss: 0.3395 - accuracy: 0.8637 - val_loss: 0.3135 - val_accuracy: 0.8724
Epoch 4/50
190/190 [==============================] - 1s 3ms/step - loss: 0.3213 - accuracy: 0.8716 - val_loss: 0.3064 - val_accuracy: 0.8744
Epoch 5/50
190/190 [==============================] - 1s 4ms/step - loss: 0.3126 - accuracy: 0.8732 - val_loss: 0.2972 - val_accuracy: 0.8804
Epoch 6/50
190/190 [==============================] - 1s 3ms/step - loss: 0.3069 - accuracy: 0.8777 - val_loss: 0.2925 - val_accuracy: 0.8830
Epoch 7/50
190/190 [==============================] - 1s 3ms/step - loss: 0.2970 - accuracy: 0.8820 - val_loss: 0.2872 - val_accuracy: 0.8830
Epoch 8/50
190/1

In [40]:
loss, accuracy = model.evaluate(X_test, y_test)

60/60 [==============================] - 0s 2ms/step - loss: 0.2689 - accuracy: 0.8995


In [41]:
print("Acurácia no teste:", accuracy)

Acurácia no teste: 0.8995240330696106


In [42]:
y_prob = model.predict(X_test).flatten()

60/60 [==============================] - 0s 1ms/step


In [43]:
threshold = 0.7
clientes_alvo = np.where(y_prob > threshold)[0]
print(f"Clientes com probabilidade > {threshold*100}%:", clientes_alvo)

Clientes com probabilidade > 70.0%: [  11   22   33   45   66   94  111  129  130  139  146  167  170  182
  184  193  198  209  216  225  226  243  246  263  270  271  304  310
  329  359  383  385  395  398  400  406  409  425  426  434  448  450
  454  456  465  466  475  479  513  532  536  546  551  565  589  622
  630  674  678  679  682  683  699  701  718  723  728  751  759  766
  792  795  810  816  836  857  870  875  883  897  930  933  942  959
  970  971  973  985  986 1015 1033 1039 1046 1065 1067 1068 1087 1089
 1092 1110 1120 1129 1136 1138 1161 1164 1166 1170 1174 1196 1201 1212
 1238 1243 1244 1261 1263 1278 1291 1294 1312 1317 1320 1329 1338 1340
 1365 1367 1372 1379 1380 1415 1419 1427 1434 1443 1469 1478 1503 1544
 1545 1552 1561 1565 1580 1587 1594 1609 1613 1627 1634 1639 1649 1654
 1661 1667 1668 1681 1682 1684 1698 1706 1709 1710 1722 1728 1730 1775
 1788 1795 1818 1835 1838 1841 1847 1849 1865 1884 1888]


In [45]:
model.save("rnn_churn_model.keras")